<a href="https://colab.research.google.com/github/Toan02Ky-UIT/CodeProject/blob/main/model_chinh_sua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import json
import re
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
df = pd.read_csv('/gdrive/MyDrive/Project/Bai2/annonimized.csv')


In [ ]:
df = df.rename(columns={
    "concat('it001',`assignment_id`)": "assignment_id",
    "concat('it001',`problem_id`)": "problem_id",
    "concat('it001', username)": "username",
    "concat('it001',`language_id`)": "language_id"
})


In [ ]:
df['created_dt'] = pd.to_datetime('2024-' + df['created_at'], format='%Y-%d-%m %H:%M:%S', errors='coerce')
df['day_of_year'] = df['created_dt'].dt.dayofyear
df['hour'] = df['created_dt'].dt.hour + df['created_dt'].dt.minute/60

In [ ]:
df['created_at'] = df['created_at'].astype(str)

df['day_month'] = df['created_at'].str.extract(r'(\d{2}-\d{2})')[0]

def day_of_year(day_month):
    try:
        if pd.isna(day_month):
            return None
        month, day = map(int, day_month.split('-'))
        days_in_months = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]
        return days_in_months[month - 1] + day
    except:
        return None


df['day_in_year'] = df['day_month'].apply(day_of_year)


In [ ]:
df['has_fatal_error'] = df['judgement'].str.contains('fatal error', case=False, na=False).astype(int)
df['verdict_WRONG_count'] = df['judgement'].str.count('WRONG', flags=re.IGNORECASE)
df['time_limit_exceeded_count'] = df['judgement'].str.count('Time Limit Exceeded', flags=re.IGNORECASE)


In [ ]:
df['final_score'] = df['pre_score'] * df['coefficient'] / 10000

le_lang = LabelEncoder()
df['language_encoded'] = le_lang.fit_transform(df['language_id'].astype(str))

In [ ]:
features = df.groupby('username').agg(
    total_submissions=('problem_id', 'count'),
    distinct_assignments=('assignment_id', 'nunique'),
    distinct_problems=('problem_id', 'nunique'),
    final_submission_count=('is_final', 'sum'),
    mean_final_score=('final_score', 'mean'),
    max_score_problem_count=('final_score', lambda x: (x == 10).sum()),
    compilation_error_rate=('status', lambda x: (x == 'Compilation Error').mean()),
    syntax_error_rate=('status', lambda x: (x == 'Syntax Error').mean()),
    pending_rate=('status', lambda x: (x == 'Pending').mean()),
    has_fatal_error=('has_fatal_error', 'max'),
    verdict_WRONG_count=('verdict_WRONG_count', 'sum'),
    time_limit_exceeded_count=('time_limit_exceeded_count', 'sum'),
    language_count=('language_id', 'nunique'),
    dominant_language=('language_encoded', lambda x: x.mode().iloc[0] if not x.mode().empty else -1),
    mean_hour=('hour', 'mean'),
    std_hour=('hour', 'std'),
).reset_index()

time_range = df.groupby('username')['day_of_year'].agg(['min', 'max']).reset_index()
time_range['active_range_days'] = time_range['max'] - time_range['min']
time_range.loc[time_range['active_range_days'] < 0, 'active_range_days'] += 365

features = features.merge(time_range[['username', 'active_range_days']], on='username', how='left')


In [ ]:
assign_count = df.groupby('username')['assignment_id'].nunique().rename('distinct_assignments')

prob_count = df.groupby('username')['problem_id'].nunique().rename('distinct_problems_total')

last_sub = df.sort_values(['username', 'problem_id', 'created_at']).groupby(['username', 'problem_id']).tail(1)
final_prob_score = last_sub.groupby('username')['final_score'].mean().rename('mean_final_score_last_attempt')

max_score_ratio = last_sub.groupby('username').apply(lambda x: (x['final_score'] == 10).mean()).rename('max_score_ratio_last')

first_last_time = df.groupby('username').agg(
    first_time=('day_in_year', 'min'),
    last_time=('day_in_year', 'max')
)
first_last_time['active_days'] = first_last_time['last_time'] - first_last_time['first_time']
first_last_time['active_days'] = first_last_time['active_days'].apply(lambda x: x + 365 if x < 0 else x)

subs_per_problem = df.groupby(['username', 'problem_id']).size().groupby('username').mean().rename('mean_submissions_per_problem')

most_lang = df.groupby(['username', 'language_id']).size().groupby('username').idxmax().apply(lambda x: x[1])
df = df.merge(most_lang.rename('dominant_lang'), on='username', how='left')
df['diff_lang'] = (df['language_id'] != df['dominant_lang']).astype(int)
diff_lang_rate = df.groupby('username')['diff_lang'].mean().rename('diff_lang_rate')

time_diff_mean_per_prob = df.sort_values(['username', 'problem_id', 'day_of_year']).groupby(['username', 'problem_id']).day_of_year.diff()
time_diff_mean_per_prob = time_diff_mean_per_prob.apply(lambda x: x + 365 if x < 0 else x)
time_diff_mean_per_user = time_diff_mean_per_prob.groupby(df['username']).mean().rename('mean_time_diff_all')

/tmp/ipython-input-197-895930807.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  max_score_ratio = last_sub.groupby('username').apply(lambda x: (x['final_score'] == 10).mean()).rename('max_score_ratio_last')


In [ ]:
features = features.merge(assign_count, on='username', how='left') \
                       .merge(prob_count, on='username', how='left') \
                       .merge(final_prob_score, on='username', how='left') \
                       .merge(max_score_ratio, on='username', how='left') \
                       .merge(first_last_time[['active_days']], on='username', how='left') \
                       .merge(subs_per_problem, on='username', how='left') \
                       .merge(diff_lang_rate, on='username', how='left') \
                       .merge(time_diff_mean_per_user, on='username', how='left')


In [ ]:
diemqt = pd.read_csv('/gdrive/MyDrive/Project/Bai2/qt-public.csv')
diemqt.columns = ['username', 'diemqt']

In [ ]:
full_df = features.merge(diemqt, on='username', how='left')

full_df['diemqt'] = full_df['diemqt'].astype(str).str.replace('\xa0', '', regex=True).str.strip()
full_df['diemqt'] = pd.to_numeric(full_df['diemqt'], errors='coerce')

# B10. Tách tập có và không có điểm quá trình
train_df = full_df[full_df['diemqt'].notna()]
predict_df = full_df[full_df['diemqt'].isna()]

# B11. Train-test split và chuẩn hóa
X = train_df.drop(columns=['username', 'diemqt'])
y = train_df['diemqt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = RandomForestRegressor(random_state=42, n_estimators=200)
model.fit(X_train, y_train)

# B13. Đánh giá
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R² trên tập test: {r2:.4f}')


R² trên tập test: 0.2448


In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [ ]:
cat_model = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    verbose=0,
    random_seed=42
)

cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)
r2_cat = r2_score(y_test, y_pred_cat)
print(f'R² CatBoost: {r2_cat:.4f}')

lgb_model = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    random_state=42
)

lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
r2_lgb = r2_score(y_test, y_pred_lgb)
print(f'R² LightGBM: {r2_lgb:.4f}')

R² CatBoost: 0.2617
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000126 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2275
[LightGBM] [Info] Number of data points in the train set: 603, number of used features: 22
[LightGBM] [Info] Start training from score 8.242952
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

In [ ]:
avg_pred = (y_pred + y_pred_cat + y_pred_lgb) / 3

r2 = r2_score(y_test, avg_pred)
print(f"R² của Averaging: {r2:.4f}")

R² của Averaging: 0.2838
